In [16]:
import pandas as pd
import itertools as it

In [26]:
ingest = pd.read_csv('full_players.csv', index_col=0).reset_index(drop=True)
schedule = pd.read_csv('2021_schedule', index_col=0) 

ingest = ingest[ingest['Name'].isin(ingest_2021['Name'].unique())]

schedule = schedule.append(schedule.rename({'Team 1':"Team 2", "Team 2":"Team 1"},axis=1))
ingest_2021 = schedule.merge(ingest_2021[['Team 1', 'Team 2', 'Position', 'Name', 'Score', 'Price']], on=['Team 1','Team 2'])


ingest_2021 = schedule.merge(ingest_2021[['Name', 'Team 1', 'Position', 'Price']], on=['Team 1','Team 2'])

ingest['Score'] = pd.to_numeric(ingest['Score'].replace('-',0))
ingest['Score'] = ingest[['Name','Team 1', 'Team 2', 'Position','Score']].groupby(['Name','Team 1', 'Team 2', 'Position']).sum()

names = ingest['Name'].sort_values().unique()
rounds = ingest['Round'].sort_values().unique()
players = pd.DataFrame(list(it.product(names,rounds)),columns=['Name','Round'])
players = players.merge(ingest[['Name','Team 1', 'Price']].drop_duplicates(), on = ['Name']).merge(ingest[['Name', 'Round','Score']].drop_duplicates(), how='left', on=['Name', 'Round'])


KeyError: 'Round'

In [18]:
players

,Name,Round,Team 1,Price,Score
0,Aaron Francis,1,Bombers,387000,15.333333
1,Aaron Francis,2,Bombers,387000,20.600000
2,Aaron Francis,3,Bombers,387000,NaN
3,Aaron Francis,4,Bombers,387000,15.000000
4,Aaron Francis,5,Bombers,387000,7.400000
...,...,...,...,...,...
11011,Zak Jones,16,Saints,708000,56.285714
11012,Zak Jones,17,Saints,708000,51.000000
11013,Zak Jones,18,Saints,708000,24.571429
11014,Zak Jones,18,Saints,708000,31.785714


In [23]:
len(
    
)*len(players['Round'].unique())-len(players)

-612